西瓜书习题4.3：试编程实现基于信息熵进行划分选择的决策树算法，并为表 4.3 中数据生成一棵决策树。

In [1]:
# -*- coding: utf-8 -*
import numpy as np
import pandas as pd


In [2]:
df = pd.read_csv('watermelon3_0_En.csv', encoding = "utf-8")
df = df.drop(['No.','Density','SugerRatio'],axis=1)
print(df)

    Color       Root   Knocks   Texture Umbilicus Touch  Label
0   green       curl  heavily  distinct    sunken  hard      1
1   black       curl     dull  distinct    sunken  hard      1
2   black       curl  heavily  distinct    sunken  hard      1
3   green       curl     dull  distinct    sunken  hard      1
4   white       curl  heavily  distinct    sunken  hard      1
5   green  lightCurl  heavily  distinct    dimple  soft      1
6   black  lightCurl  heavily      blur    dimple  soft      1
7   black  lightCurl  heavily  distinct    dimple  hard      1
8   black  lightCurl     dull      blur    dimple  hard      0
9   green      stiff    clear  distinct    smooth  soft      0
10  white      stiff    clear     fuzzy    smooth  hard      0
11  white       curl  heavily     fuzzy    smooth  soft      0
12  green  lightCurl  heavily      blur    sunken  hard      0
13  white  lightCurl     dull      blur    sunken  hard      0
14  black  lightCurl  heavily  distinct    dimple  soft

In [3]:
'''
结点类：

@para attribute：结点要划分的属性
@para label：结点的类别
@para attribute_next：下一个结点
'''
class Node(object):
    def __init__(self, attribute=None, label=None, node_next={}):
        self.attribute = attribute
        self.label = label
        self.node_next = node_next
    

In [4]:
'''
统计数据集中样例各个类别的数量，在西瓜数据3.0中即为是和否的数量：

@para   df：数据集
@return label_count：各个类别的数量：label_count[0]:正例数量，label_count[1]反例数量
'''
def LabelCount(df):
    label_count=[]
    label_count.append(df[df['Label']==1]['Label'].count())
    label_count.append(df[df['Label']==0]['Label'].count())
    return label_count

In [5]:
'''
样本数据比较

@para    df：数据集
@return  True：表示数据集中所有样本在所有属性上取值相同
         False：数据集中又不同取值
'''
def DataCompare(df, A):
    if len(df.shape)==0 :
        return True
    a = list(attr_set.keys())
    for row in range(df.shape[0]-1): 
        com  = df[a].iloc[0] == df[a].iloc[row+1]#iloc:索引第i行；loc：索引行标签为i的行
        if(com[com==True].count()<com.count()):
            return False
    return True

In [6]:
'''
创建属性集

@para    df：数据集
@return  attr_set:属性集
                 字典格式如：{'敲声': ['浊响', '沉闷', '清脆']}
'''
def CreateAttrSet(df):
    attr_set = {}
    attr_name = df.columns
    for i in attr_name[:6]:
        attr_set[i] = df[i].unique()
    return attr_set

In [7]:
'''
挑选数据集中A属性中包含a值的样例
'''
def DataSelect(df, A, a):
    return df[df[A]==a]

基于信息增益属性选择：ID3

In [8]:
'''
计算信息熵
@para    df：数据集
@return  ent_d:数据集df的信息熵
'''
def GetEnt(df):
    ent_d = 0.0
    classes = LabelCount(df)
    class_sum = len(df)
    for k in classes:        
        if k!=0 :
            p_k = k/class_sum
            ent_d += - p_k*np.log2(p_k)
    return ent_d
'''
计算信息增益

@para    df：数据集
@para    attr:需要计算信息增益的属性
@return  gain:属性attr的信息增益
'''
def GetGain(df, attr):
    ent_d = GetEnt(df)
    gain_temp = 0.0
    for v in attr[1]:
        df_v = df[df[attr[0]]==v]#挑选数据集中attr[0]属性中包含v值的样例
        gain_temp += len(df_v)/len(df)*GetEnt(df_v)
    gain = GetEnt(df) - gain_temp
    return gain
'''
基于信息增益属性选择

@para    df：数据集
@para    attr_set:属性集
@return  best_attr:属性集中信息增益最大的属性
''' 
def AttrSelectBaseGain(df, attr_set):
    best_gain = 0.0
    best_attr = None
    for attr in attr_set.items():
        attr_gain = GetGain(df, attr)
        if attr_gain>best_gain :
            best_gain = attr_gain
            best_attr = attr
    return best_attr

基于增益率属性选择：C4.5

In [9]:
'''
计算增益率中的属性固有值

@para    df：数据集
@para    att:需要计算增益率的属性
@return  iv_attr:属性attr的增益率
'''
def IV(df, attr):
    iv_attr = 0.0 #属性attr的固有值
    for v in attr[1]:
        df_v = df[df[attr[0]]==v]#挑选数据集中attr[0]属性中包含v值的样例
        if len(df_v)!=0 :
            iv_attr -= len(df_v)/len(df) * np.log2(len(df_v)/len(df))
    return iv_attr
'''
基于增益率属性选择

@para    df：数据集
@para    attr_set:属性集
@return  best_attr:属性集中增益率最大的属性
'''
def AttrSelectBaseGainRatio(df,attr_set):
    best_gain_ratio = 0.0
    best_attr = None
    for attr in attr_set.items():
        gain_ratio = GetGain(df, attr)/IV(df, attr)
        if gain_ratio>best_gain_ratio :
            best_gain_ratio = gain_ratio
            best_attr = attr
    return best_attr

基于基尼指数属性选择：CART决策树

In [37]:
'''
计算基尼值:西瓜书公式4.5

@para    df：数据集
@return  gini:数据集df的基尼值
'''
def GetGini(df):
    label_count = LabelCount(df)
    gini_temp = 0.0
    for k in label_count:
        if len(df)!=0:
            gini_temp += np.square(k/len(df))
    gini = 1-gini_temp
    print('基尼值：',gini)
    return gini
'''
计算属性attr的基尼指数：西瓜书公式4.6

@para    df：数据集
@para    attr:属性
@return  gini_index:属性attr的基尼指数
'''
def GetGini_index(df,attr):
    gini_index = 0.0
    for v in attr[1]:
        df_v = df[df[attr[0]]==v]
        gini_index += len(df_v)/len(df)*GetGini(df_v)
    print('基尼指数',gini_index)
    return gini_index
'''
基于基尼指数属性选择

@para    df：数据集
@para    attr_set:属性集
@return  best_attr:属性集中基尼指数最小的属性
'''
def AttrSelectBaseGainIndex(df,attr_set):
    best_attr = None
    best_gini_index = 100
    for attr in attr_set.items():
        gini_index = GetGini_index(df,attr)
        
        if best_gini_index>=gini_index:
            best_gini_index = gini_index
            best_attr = attr
    print(best_gini_index)
    return best_attr

In [11]:
'''
测试决策树绘图
'''

def DrawPNG(root, out_file):
    '''
    visualization of decision tree from root.
    @param root: Node, the root node for tree.
    @param out_file: str, name and path of output file
    '''
    try:
        from pydotplus import graphviz
    except ImportError:
        print("module pydotplus.graphviz not found")
        
    g = graphviz.Dot()  # generation of new dot   

    TreeToGraph(0, g, root)
    g2 = graphviz.graph_from_dot_data( g.to_string() )
                                                                                            
    g2.write_png(out_file) 

def TreeToGraph(i, g, root):
    '''
    build a graph from root on
    @param i: node number in this tree
    @param g: pydotplus.graphviz.Dot() object
    @param root: the root node
    
    @return i: node number after modified  
#   @return g: pydotplus.graphviz.Dot() object after modified
    @return g_node: the current root node in graphviz
    '''
    try:
        from pydotplus import graphviz
    except ImportError:
        print("module pydotplus.graphviz not found")
    
    if root.attribute == None:
        g_node_label = "Node:%d\nLabel:%s" % (i, root.label)
    else:
        g_node_label = "Node:%d\nAttr:%s" % (i, root.attribute)
    g_node = i
    g.add_node( graphviz.Node( g_node, label = g_node_label ))
    
    for value in list(root.node_next):
        i, g_child = TreeToGraph(i+1, g, root.node_next[value])
        g.add_edge( graphviz.Edge(g_node, g_child, label = value) ) 

    return i, g_node

In [20]:
'''
创建决策树
'''
def CreateDecisionTree(data_set, attr_set):
    node = Node(None, None, {})
    label_count = LabelCount(data_set)
    #情形1：数据集中包含的样本全属于同一类时，递归结束，返回
    if(label_count[0]==data_set['Label'].count()):
        node.label = 1
        return node
    if(label_count[1]==data_set['Label'].count()):
        node.label = 0
        return node
    #情形2：属性集为空或样本在所有属性上取值相同，无法划分
    if(len(attr_set)==0 or DataCompare(data_set, attr_set)==True):
        if(label_count[0]>label_count[1]):
            node.label = 1
        else:
            node.label = 0
        return node
    
    #划分选择，挑选最优的属性作为当前结点
    best_attr = AttrSelectBaseGainIndex(data_set,attr_set)
    print(data_set)
    print(attr_set)
    node.attribute = best_attr[0]
    for i in best_attr[1]:
        node.node_next[i]=Node(None, Node, {})
        data_set_a = data_set[data_set[best_attr[0]]==i]#挑选数据集中best_attr[0]属性中包含i值的样例
        
        if(data_set_a.empty):
            if(label_count[0]>label_count[1]):
                node.node_next[i].label = 1
            else:
                node.node_next[i].label = 0
            return node
        else:
            attr_set.pop(best_attr[0],best_attr[1])
            node.node_next[i] = CreateDecisionTree(data_set_a, attr_set)  
    return node


In [38]:
attr_set = CreateAttrSet(df)
root = CreateDecisionTree(df, attr_set)


基尼值： 0.5
基尼值： 0.4444444444444444
基尼值： 0.31999999999999984
基尼指数 0.42745098039215684
基尼值： 0.345679012345679
基尼值： 0.31999999999999984
基尼值： 0.0
基尼指数 0.2771241830065359
基尼值： 0.40816326530612246
基尼值： 0.5
基尼值： 0.0
基尼指数 0.3445378151260504
基尼值： 0.5
基尼值： 0.48
基尼指数 0.49411764705882355
基尼值： 0.46875
基尼值： 0.48979591836734704
基尼值： 0.0
基尼指数 0.42226890756302526
基尼值： 0.48
基尼值： 0.48
基尼值： 0.0
基尼指数 0.4235294117647059
0.2771241830065359
    Color       Root   Knocks   Texture Umbilicus Touch  Label
0   green       curl  heavily  distinct    sunken  hard      1
1   black       curl     dull  distinct    sunken  hard      1
2   black       curl  heavily  distinct    sunken  hard      1
3   green       curl     dull  distinct    sunken  hard      1
4   white       curl  heavily  distinct    sunken  hard      1
5   green  lightCurl  heavily  distinct    dimple  soft      1
6   black  lightCurl  heavily      blur    dimple  soft      1
7   black  lightCurl  heavily  distinct    dimple  hard      1
8   black  lig

In [39]:
DrawPNG(root, "decision_tree_ID3.png")

In [ ]:
def PredictTree(predict_data,node):
    if(node.label!=None):
        return node.label
    attr_val = predict_data[node.attribute]    
    return PredictTree(predict_data,node.node_next[attr_val])

In [ ]:
a = PredictTree(df.iloc[16],root)
print(a)